In [103]:
import sys
sys.path.insert(0, '..') # in order to be able to import from scripts.py

In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [105]:
from subplots_from_axsize import subplots_from_axsize

def plot_result(outfile, result, title=None):
    data = result.states
    
    fig, ax = subplots_from_axsize(
        1, 1, axsize=(20, 8),
        left=0., right=0., bottom=0., top=0.
    )

    data_selected = data.copy()
    data_selected['E'] = data_selected['E'] > 0
    data_selected['I'] = data_selected['I'] > 0
    data_selected['R'] = data_selected['R'] > 0

    img_E = data_selected.groupby(['seconds', 'h'])['E'].mean().unstack().to_numpy().T
    img_I = data_selected.groupby(['seconds', 'h'])['I'].mean().unstack().to_numpy().T
    img_R = data_selected.groupby(['seconds', 'h'])['R'].mean().unstack().to_numpy().T

    img = img_E + img_I

    ax.imshow(
        img,
        cmap='gray',
        origin='lower',
        aspect='auto',
        interpolation='none',
    )

    ax.set_xlabel('time')
    ax.set_axis_off()
    
    if title is not None:
        ax.annotate(
            title, (0.5, 0.9),
            xycoords='axes fraction', va='center', ha='center',
            color='red', fontsize=32
        )

    fig.savefig(outfile)
    plt.close(fig)

In [106]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

In [ ]:
sim_time = 5000

for channel_height in [6, 7 , 8]:

    client = VisAVisClient(
    )

    for interval in [50, 100, 150, 200]:

        protocol_file_path = make_protocol(
            pulse_intervals =(sim_time // interval) * [interval],
            duration=4,
            out_folder='./'
        )
        
        result = client.run(
            parameters_json=PARAMETERS_DEFAULT,
            protocol_file_path= protocol_file_path,
        )
        plot_result(
            f"./tkankony_{channel_height}_{interval}.png",
            result,
            title=f"channel height: {channel_height}, interval: {interval}"
        )